# MMOE
## 曝光 → 转化

In [2]:
import datetime
import numpy as np
import pandas as pd
import joblib
import warnings
import logging
import os
import gc
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import collections
import re
import copy
import torch
import shap
import utils.utils as util

import utils_

from functools import reduce
from tqdm import tqdm
from dateutil.relativedelta import relativedelta
from joblib import Parallel, delayed
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.inspection import permutation_importance
from model.mmoe_condition_2 import MMOE
from torch.utils.data import DataLoader
from utils.dataset import DatasetLoader, DatasetLoader_w, DatasetLoader_www
from utils.warmup_lr import GradualWarmupScheduler
from loss.BCE_weighted_multi_task import BCEWL_weighted_multi_task


# pd.set_option('display.max_columns', None)
# pd.set_option('max_row', 500)
warnings.filterwarnings('ignore')
tqdm.pandas(desc='pandas bar')

In [ ]:
torch.__version__

## 数据处理

In [ ]:
df_fusion = utils_.load_pickle('../data/exp2suc/')
df_asp1 = utils_.load_pickle('../data/exp2suc/')
df_asp2 = utils_.load_pickle('../data/exp2suc/')

print(df_fusion.shape)
print(df_asp1.shape)
print(df_asp2.shape)

In [ ]:
list_feats_id_dt_item_y = ['uid', 'obs_dt', 'dt', 'item', 'table_type', 'label_apply', 'label_submit', 'label_pre_pass', 'label_pass', 'label_pass_1']

list_feats_x_1 = [x for x in 
                  utils_.load_pickle('../data/exp2suc/list_feats/list_feats_x_fusion_pi_20230716_20231001.pickle') 
                  if x not in 
                  ['xxx']
                 ]
list_feats_x_2 = [x for x in 
                  utils_.load_pickle('../data/exp2suc/list_feats/list_feats_x_asp1_pi_20230716_20231001.pickle') 
                  if x not in 
                  []
                 ]
list_feats_x_3 = [x for x in 
                  utils_.load_pickle('../data/exp2suc/list_feats/list_feats_x_asp2_pi_20230716_20231001.pickle') 
                  if x not in 
                  []
                 ]

print(len(list_feats_x_1))
print(len(list_feats_x_2))
print(len(list_feats_x_3))

In [ ]:
df = df_fusion[list_feats_id_dt_item_y+list_feats_x_1].\
    merge(df_asp1[list_feats_id_dt_item_y+list_feats_x_2]).\
    merge(df_asp2[list_feats_id_dt_item_y+list_feats_x_3])

print(df.shape)
df.head()

In [ ]:
df_des = utils_.df_des(df)
df_des.to_csv('../data/exp2suc/loss_pp_cb/', encoding='utf-8')

df_des[df_des['Miss Percent(%)']>0]

In [ ]:
df[['item', 'table_type', 'uid']].groupby(by=['item', 'table_type']).count()

In [ ]:
dict_item = {
    'aaa': 0, 
    'bbb': 1, 
    'ccc': 2, 
    'ddd': 3, 
    'eee': 4, 
    'fff': 5
}

df['item_id'] = df['item']
df['item_id'].replace(dict_item, inplace=True)
df.head()

In [ ]:
df[['item', 'item_id', 'table_type', 'uid']].groupby(['item', 'item_id', 'table_type']).count()

In [ ]:
# 物料\表类型特征处理
# one hot encoding
list_feats_ohe = ['item_id', 'table_type']
list_df_ohe = []
try:
    with tqdm(list_feats_ohe) as t:
        for feat in t:
            df_ohe_feat = utils_.one_hot_encoder(df, feat)
            list_df_ohe.append(df_ohe_feat)
except KeyboardInterrupt:
    t.close()
    raise
t.close()

df_ohe_part = pd.concat(list_df_ohe, axis=1)
print(df_ohe_part.shape)
df_ohe_part.head()

In [ ]:
df_ohe = pd.concat([df, df_ohe_part], axis=1)
print(df_ohe.shape)
df_ohe.head()

In [ ]:
utils_.save_pickle(df_ohe, '../data/exp2suc/loss_pp_cb/')

* 模型小物料 batch loss 权重调整（aa短表ys、小物料长表）
* 6物料，aa、bb、cc、dd、ee、ff
* fusion & aspiration；长表，“曝光→hpyk”特征；短表（aa），“曝光→ys通过”特征；761 维
* 2023.07.16~2023.10.01
* 训练验证集调整：
  1. aa负样本（fqjj）下采样，线上打分需校准 ✅
  2. cc负样本（fqjj）下采样，线上打分需校准 ✅
  3. dd负样本（fqjj）下采样，线上打分需校准 ✅

In [ ]:
df_train = utils_.load_pickle('../data/exp2suc/df_pi_ohe_train_20230716_20231001.pickle')
print(df_train.shape)
df_train.head()

In [ ]:
# 划分训练集&验证集，8:2，80%
df_train_train, df_train_eval = train_test_split(df_train, test_size=0.2, random_state=2023)
print(df_train_train.shape)
print(df_train_eval.shape)

In [ ]:
list_feats_item_item = [x for x in 
                        df_train.columns 
                        if x.startswith('item_id_') 
                        and x not in 
                        []
                       ]
list_feats_item_table = [x for x in 
                         df_train.columns 
                         if x.startswith('table_type_') 
                         and x not in 
                         []
                        ]
list_feats_x_1 = [x for x in 
                  utils_.load_pickle('../data/exp2suc/list_feats/list_feats_x_fusion_pi_20230716_20231001_new.pickle') 
                  if x not in 
                  ['xxx']
                 ]
list_feats_x_2 = [x for x in 
                  utils_.load_pickle('../data/exp2suc/list_feats/list_feats_x_asp1_pi_20230716_20231001_new.pickle') 
                  if x not in 
                  []
                 ]
list_feats_x_3 = [x for x in 
                  utils_.load_pickle('../data/exp2suc/list_feats/list_feats_x_asp2_pi_20230716_20231001_new.pickle') 
                  if x not in 
                  []
                 ]

df_train_train_id = df_train_train[['uid', 'obs_dt', 'dt', 'item', 'item_id', 'table_type']]
df_train_train_y = df_train_train[['label_apply', 'label_submit', 'label_pre_pass', 'label_pass', 'label_pass_1']]
df_train_train_X = df_train_train[
    list_feats_item_item+
    list_feats_item_table+
    list_feats_x_1+
    list_feats_x_2+
    list_feats_x_3
]
print(df_train_train_id.shape)
print(df_train_train_y.shape)
print(df_train_train_X.shape)

df_train_eval_id = df_train_eval[['uid', 'obs_dt', 'dt', 'item', 'item_id', 'table_type']]
df_train_eval_y = df_train_eval[['label_apply', 'label_submit', 'label_pre_pass', 'label_pass', 'label_pass_1']]
df_train_eval_X = df_train_eval[
    list_feats_item_item+
    list_feats_item_table+
    list_feats_x_1+
    list_feats_x_2+
    list_feats_x_3
]
print(df_train_eval_id.shape)
print(df_train_eval_y.shape)
print(df_train_eval_X.shape)

In [ ]:
utils_.save_pickle(df_train_train_id, '../data/exp2suc/loss_pp_cb_feats/df_id_train_train_20230716_20231001.pickle')
utils_.save_pickle(df_train_train_y, '../data/exp2suc/loss_pp_cb_feats/df_y_train_train_20230716_20231001.pickle')
utils_.save_pickle(df_train_train_X, '../data/exp2suc/loss_pp_cb_feats/df_X_pi_train_train_20230716_20231001.pickle')

utils_.save_pickle(df_train_eval_id, '../data/exp2suc/loss_pp_cb_feats/df_id_train_eval_20230716_20231001.pickle')
utils_.save_pickle(df_train_eval_y, '../data/exp2suc/loss_pp_cb_feats/df_y_train_eval_20230716_20231001.pickle')
utils_.save_pickle(df_train_eval_X, '../data/exp2suc/loss_pp_cb_feats/df_X_pi_train_eval_20230716_20231001.pickle')

## 入参处理

* 训练集

In [ ]:
# df_train_train_id = utils_.load_pickle('../data/exp2suc/loss_pp_cb/df_id_train_train_20230716_20231001.pickle')
# df_train_train_y = utils_.load_pickle('../data/exp2suc/loss_pp_cb/df_y_train_train_20230716_20231001.pickle')
# df_train_train_X = utils_.load_pickle('../data/exp2suc/loss_pp_cb/df_X_pi_train_train_20230716_20231001.pickle')

df_train_train_id = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/df_id_train_train_20230716_20231001.pickle')
df_train_train_y = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/df_y_train_train_20230716_20231001.pickle')
df_train_train_X = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/df_X_pi_train_train_20230716_20231001.pickle')

print(df_train_train_id.shape)
print(df_train_train_y.shape)
print(df_train_train_X.shape)

In [ ]:
condition_feats_item = ['item_id_{}'.format(x) for x in range(6)]

condition_feats_table = ['table_type_{}'.format(x) for x in range(1, 3)]

sparse_feats = [
    'xxx'
]
sparse_feats = [x for x in df_train_train_X.columns if x in sparse_feats]

dense_feats = [x for x in df_train_train_X.columns 
               if x not in condition_feats_item+condition_feats_table+sparse_feats]

print(len(condition_feats_item))
print(len(condition_feats_table))
print(len(sparse_feats))
print(len(dense_feats))

In [ ]:
for x in sparse_feats:
    print('{}:{}'.format(x, df_train_train_X[x].nunique()))

In [ ]:
sparse_feats = [x for x in sparse_feats if df_train_train_X[x].nunique()<=32 
                and x not in ['xxx']]
print(len(sparse_feats))
sparse_feats

In [ ]:
dense_feats = [x for x in df_train_train_X.columns 
               if x not in condition_feats_item+condition_feats_table+sparse_feats]
print(len(dense_feats))

In [ ]:
# utils_.save_pickle(condition_feats_item, '../data/exp2suc/loss_pp_cb/list_condition_feats_item.pickle')
# utils_.save_pickle(condition_feats_table, '../data/exp2suc/loss_pp_cb/list_condition_feats_table.pickle')
# utils_.save_pickle(sparse_feats, '../data/exp2suc/loss_pp_cb/list_sparse_feats.pickle')
# utils_.save_pickle(dense_feats, '../data/exp2suc/loss_pp_cb/list_dense_feats.pickle')

utils_.save_pickle(condition_feats_item, '../data/exp2suc/loss_pp_cb_feats/list_condition_feats_item.pickle')
utils_.save_pickle(condition_feats_table, '../data/exp2suc/loss_pp_cb_feats/list_condition_feats_table.pickle')
utils_.save_pickle(sparse_feats, '../data/exp2suc/loss_pp_cb_feats/list_sparse_feats.pickle')
utils_.save_pickle(dense_feats, '../data/exp2suc/loss_pp_cb_feats/list_dense_feats.pickle')

In [ ]:
# 连续
ss = StandardScaler()
X_train_train_dense_ss = ss.fit_transform(df_train_train_X[dense_feats])

# joblib.dump(ss, '../data/exp2suc/loss_pp_cb/ss.pickle')
joblib.dump(ss, '../data/exp2suc/loss_pp_cb_feats/ss.pickle')

In [ ]:
df_X_train_train_dense = pd.DataFrame(X_train_train_dense_ss, columns=dense_feats)
print(df_X_train_train_dense.shape)
df_X_train_train_dense.head()

In [ ]:
# 离散
dict_lbe = {}
list_X_train_train_sparse = []

try:
    with tqdm(sparse_feats) as t:
        for x in t:
            lbe = LabelEncoder()
            df_X_sparse_each = pd.DataFrame(lbe.fit_transform(df_train_train_X[x]), columns=[x])
            dict_lbe[x] = lbe
            list_X_train_train_sparse.append(df_X_sparse_each)
except KeyboardInterrupt:
    t.close()
    raise
t.close()

# joblib.dump(dict_lbe, '../data/exp2suc/loss_pp_cb/dict_lbe.pickle')
joblib.dump(dict_lbe, '../data/exp2suc/loss_pp_cb_feats/dict_lbe.pickle')

df_X_train_train_sparse = pd.concat(list_X_train_train_sparse, axis=1)
print(df_X_train_train_sparse.shape)
df_X_train_train_sparse.head()

In [ ]:
# 合并
df_train_train_X_condition_item = df_train_train_X[condition_feats_item].reset_index(drop=True)
df_train_train_X_condition_table = df_train_train_X[condition_feats_table].reset_index(drop=True)
df_train_train_X_transform = pd.concat([df_train_train_X_condition_item, df_train_train_X_condition_table, 
                                        df_X_train_train_sparse, df_X_train_train_dense], axis=1)
print(df_train_train_X_transform.shape)
df_train_train_X_transform.head()

In [ ]:
df_train_train_X.head()

In [ ]:
df_train_train_id.head()

In [ ]:
# utils_.save_pickle(df_train_train_X_transform, '../data/exp2suc/loss_pp_cb/df_X_pi_train_train_transform_20230716_20231001.pickle')
utils_.save_pickle(df_train_train_X_transform, '../data/exp2suc/loss_pp_cb_feats/df_X_pi_train_train_transform_20230716_20231001.pickle')

* 验证集

In [ ]:
# df_train_eval_id = utils_.load_pickle('../data/exp2suc/loss_pp_cb/df_id_train_eval_20230716_20231001.pickle')
# df_train_eval_y = utils_.load_pickle('../data/exp2suc/loss_pp_cb/df_y_train_eval_20230716_20231001.pickle')
# df_train_eval_X = utils_.load_pickle('../data/exp2suc/loss_pp_cb/df_X_pi_train_eval_20230716_20231001.pickle')

df_train_eval_id = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/df_id_train_eval_20230716_20231001.pickle')
df_train_eval_y = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/df_y_train_eval_20230716_20231001.pickle')
df_train_eval_X = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/df_X_pi_train_eval_20230716_20231001.pickle')

print(df_train_eval_id.shape)
print(df_train_eval_y.shape)
print(df_train_eval_X.shape)

In [ ]:
# condition_feats_item = utils_.load_pickle('../data/exp2suc/loss_pp_cb/list_condition_feats_item.pickle')
# condition_feats_table = utils_.load_pickle('../data/exp2suc/loss_pp_cb/list_condition_feats_table.pickle')
# sparse_feats = utils_.load_pickle('../data/exp2suc/loss_pp_cb/list_sparse_feats.pickle')
# dense_feats = utils_.load_pickle('../data/exp2suc/loss_pp_cb/list_dense_feats.pickle')

condition_feats_item = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/list_condition_feats_item.pickle')
condition_feats_table = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/list_condition_feats_table.pickle')
sparse_feats = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/list_sparse_feats.pickle')
dense_feats = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/list_dense_feats.pickle')

print(len(condition_feats_item))
print(len(condition_feats_table))
print(len(sparse_feats))
print(len(dense_feats))

In [ ]:
# 连续
# ss = joblib.load('../data/exp2suc/loss_pp_cb/ss.pickle')
ss = joblib.load('../data/exp2suc/loss_pp_cb_feats/ss.pickle')

X_train_eval_dense_ss = ss.transform(df_train_eval_X[dense_feats])
print(X_train_eval_dense_ss.shape)

In [ ]:
df_X_train_eval_dense_ss = pd.DataFrame(X_train_eval_dense_ss, columns=dense_feats)
print(df_X_train_eval_dense_ss.shape)
df_X_train_eval_dense_ss.head()

In [ ]:
# 离散
# dict_lbe = joblib.load('../data/exp2suc/loss_pp_cb/dict_lbe.pickle')
dict_lbe = joblib.load('../data/exp2suc/loss_pp_cb_feats/dict_lbe.pickle')

for x in sparse_feats:
    print(x, dict_lbe[x].classes_)

In [ ]:
list_X_sparse = []

try:
    with tqdm(sparse_feats) as t:
        for x in t:
            list_feat_values_unseen = list(set(df_train_eval_X[x].unique())-set(dict_lbe[x].classes_))
            if len(list_feat_values_unseen) > 0:
                print(x)
                df_train_eval_X[x].replace(list_feat_values_unseen, -1, inplace=True)
            df_train_eval_X_sparse_each = pd.DataFrame(dict_lbe[x].transform(df_train_eval_X[x]), columns=[x])
            list_X_sparse.append(df_train_eval_X_sparse_each)
except KeyboardInterrupt:
    t.close()
    raise
t.close()

df_X_train_eval_sparse = pd.concat(list_X_sparse, axis=1)
print(df_X_train_eval_sparse.shape)
df_X_train_eval_sparse.head()

In [ ]:
# 合并
df_train_eval_X_condition_item = df_train_eval_X[condition_feats_item].reset_index(drop=True)
df_train_eval_X_condition_table = df_train_eval_X[condition_feats_table].reset_index(drop=True)
df_train_eval_X_transform = pd.concat([df_train_eval_X_condition_item, df_train_eval_X_condition_table, 
                                       df_X_train_eval_sparse, df_X_train_eval_dense_ss], axis=1)
print(df_train_eval_X_transform.shape)
df_train_eval_X_transform.head()

In [ ]:
df_train_eval_X.head()

In [ ]:
df_train_eval_id.head()

In [ ]:
# utils_.save_pickle(df_train_eval_X_transform, '../data/exp2suc/loss_pp_cb/df_X_pi_train_eval_transform_20230716_20231001.pickle')
utils_.save_pickle(df_train_eval_X_transform, '../data/exp2suc/loss_pp_cb_feats/df_X_pi_train_eval_transform_20230716_20231001.pickle')

## 模型

In [ ]:
# df_X_train_train_transform = utils_.load_pickle('../data/exp2suc/loss_pp_cb/df_X_pi_train_train_transform_20230716_20231001.pickle')
df_X_train_train_transform = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/df_X_pi_train_train_transform_20230716_20231001.pickle')

print(df_X_train_train_transform.shape)
df_X_train_train_transform.head()

In [ ]:
# condition_feats_item = utils_.load_pickle('../data/exp2suc/loss_pp_cb/list_condition_feats_item.pickle')
# condition_feats_table = utils_.load_pickle('../data/exp2suc/loss_pp_cb/list_condition_feats_table.pickle')
# sparse_feats = utils_.load_pickle('../data/exp2suc/loss_pp_cb/list_sparse_feats.pickle')
# dense_feats = utils_.load_pickle('../data/exp2suc/loss_pp_cb/list_dense_feats.pickle')

condition_feats_item = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/list_condition_feats_item.pickle')
condition_feats_table = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/list_condition_feats_table.pickle')
sparse_feats = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/list_sparse_feats.pickle')
dense_feats = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/list_dense_feats.pickle')

print(len(condition_feats_item))
print(len(condition_feats_table))
print(len(sparse_feats))
print(len(dense_feats))

In [ ]:
feats_columns = [[util.denseFeature(feat) for feat in condition_feats_item]] + \
                [[util.denseFeature(feat) for feat in condition_feats_table]] + \
                [[util.sparseFeature(x, int(df_X_train_train_transform[x].max()+1), 4) for x in sparse_feats]] + \
                [[util.denseFeature(feat) for feat in dense_feats]]
feats_columns

In [ ]:
# utils_.save_pickle(feats_columns, '../data/exp2suc/loss_pp_cb/feats_columns.pickle')
utils_.save_pickle(feats_columns, '../data/exp2suc/loss_pp_cb_feats/feats_columns.pickle')

In [ ]:
# df_X_train_train_transform = utils_.load_pickle('../data/exp2suc/loss_pp_cb/df_X_pi_train_train_transform_20230716_20231001.pickle')
# df_y_train_train = utils_.load_pickle('../data/exp2suc/loss_pp_cb/df_y_train_train_20230716_20231001.pickle')

# df_X_train_eval_transform = utils_.load_pickle('../data/exp2suc/loss_pp_cb/df_X_pi_train_eval_transform_20230716_20231001.pickle')
# df_y_train_eval = utils_.load_pickle('../data/exp2suc/loss_pp_cb/df_y_train_eval_20230716_20231001.pickle')

df_X_train_train_transform = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/df_X_pi_train_train_transform_20230716_20231001.pickle')
df_y_train_train = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/df_y_train_train_20230716_20231001.pickle')

df_X_train_eval_transform = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/df_X_pi_train_eval_transform_20230716_20231001.pickle')
df_y_train_eval = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/df_y_train_eval_20230716_20231001.pickle')

print(df_X_train_train_transform.shape)
print(df_y_train_train.shape)

print(df_X_train_eval_transform.shape)
print(df_y_train_eval.shape)

In [ ]:
# 设置样本权重
# 小物料，前链路（fqjj、tjjj），小物料:其他=1:5
w_pre_link_train_train = df_X_train_train_transform[['item_id_4', 'item_id_5']].max(axis=1)
w_pre_link_train_train = w_pre_link_train_train.apply(lambda x: 5 if x == 0 else 1)

w_pre_link_train_eval = df_X_train_eval_transform[['item_id_4', 'item_id_5']].max(axis=1)
w_pre_link_train_eval = w_pre_link_train_eval.apply(lambda x: 5 if x == 0 else 1)

In [ ]:
w_pre_link_train_train.value_counts()

In [ ]:
w_pre_link_train_eval.value_counts()

In [ ]:
# aa短表，ys通过，aa短表:其他=10:1
w_gf_pp_train_train = df_X_train_train_transform[['item_id_0', 'table_type_1']].sum(axis=1)
w_gf_pp_train_train = w_gf_pp_train_train.apply(lambda x: 10 if x == 2 else 1)

w_gf_pp_train_eval = df_X_train_eval_transform[['item_id_0', 'table_type_1']].sum(axis=1)
w_gf_pp_train_eval = w_gf_pp_train_eval.apply(lambda x: 10 if x == 2 else 1)

# aa短表，ys通过，aa短表:其他=1:0
# w_gf_pp_train_train = df_X_train_train_transform[['item_id_0', 'table_type_1']].sum(axis=1)
# w_gf_pp_train_train = w_gf_pp_train_train.apply(lambda x: 1 if x == 2 else 0)

# w_gf_pp_train_eval = df_X_train_eval_transform[['item_id_0', 'table_type_1']].sum(axis=1)
# w_gf_pp_train_eval = w_gf_pp_train_eval.apply(lambda x: 1 if x == 2 else 0)

In [ ]:
w_gf_pp_train_train.value_counts()

In [ ]:
w_gf_pp_train_eval.value_counts()

In [ ]:
# 长表&小物料，后链路（hp、hpyk），aa短表:小物料:其他=1:2:10
w_after_link_train_train = df_X_train_train_transform[['item_id_0', 'item_id_4', 'item_id_5', 'table_type_1']].\
    apply(lambda x: 1 if x['item_id_0']==1 and x['table_type_1']==1 
                      else 2 if x['item_id_4']==1 or x['item_id_5']==1 
                             else 10, 
          axis=1)

w_after_link_train_eval = df_X_train_eval_transform[['item_id_0', 'item_id_4', 'item_id_5', 'table_type_1']].\
    apply(lambda x: 1 if x['item_id_0']==1 and x['table_type_1']==1 
                      else 2 if x['item_id_4']==1 or x['item_id_5']==1 
                             else 10, 
          axis=1)

# 长表&小物料，后链路（hp、hpyk），aa短表:小物料:其他=0:1:5
# w_after_link_train_train = df_X_train_train_transform[['item_id_0', 'item_id_4', 'item_id_5', 'table_type_1']].\
#     apply(lambda x: 0 if x['item_id_0']==1 and x['table_type_1']==1 
#                       else 1 if x['item_id_4']==1 or x['item_id_5']==1 
#                              else 5, 
#           axis=1)

# w_after_link_train_eval = df_X_train_eval_transform[['item_id_0', 'item_id_4', 'item_id_5', 'table_type_1']].\
#     apply(lambda x: 0 if x['item_id_0']==1 and x['table_type_1']==1 
#                       else 1 if x['item_id_4']==1 or x['item_id_5']==1 
#                              else 5, 
#           axis=1)

In [ ]:
w_after_link_train_train.value_counts()

In [ ]:
w_after_link_train_eval.value_counts()

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
logger = util.get_logger('')
util.seed_everything(2023)

In [ ]:
train_loader = DataLoader(
    DatasetLoader_www(df_X_train_train_transform.values, df_y_train_train.values, 
                      w_pre_link_train_train.values, w_gf_pp_train_train.values, w_after_link_train_train.values), 
    1024, shuffle=False, num_workers=8)
eval_loader = DataLoader(
    DatasetLoader_www(df_X_train_eval_transform.values, df_y_train_eval.values, 
                      w_pre_link_train_eval.values, w_gf_pp_train_eval.values, w_after_link_train_eval.values), 
    1024, shuffle=False, num_workers=8)

In [ ]:
config = {
    'Model': {
        'num_experts': 7, 
        'expert_hidden_units': [512, 256, 128], 
        'units': 64, 
        'num_tasks': 5, 
        'tower_hidden_units': [64, 32, 16], 
        'dropout': 0.5, 
        'use_bn': False
    }
}

# feats_columns = utils_.load_pickle('../data/exp2suc/loss_pp_cb/feats_columns.pickle')
feats_columns = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/feats_columns.pickle')

model = MMOE(config, feats_columns).to(device)
model

In [ ]:
(819*512+512*256+256*128+128*64)*7+819*7*5+(72*64+64*32+32*16+16*1)*5

In [ ]:
for m in model.modules():
    if isinstance(m, (torch.nn.Conv2d, torch.nn.Linear)):
        torch.nn.init.xavier_uniform_(m.weight)
        # nn.init.kaiming_uniform_(m.weight)
    elif isinstance(m, torch.nn.BatchNorm1d):
        torch.nn.init.constant_(m.weight, 1)
        torch.nn.init.constant_(m.bias, 0)

In [ ]:
def train(train_loader, eval_loader, model, optimizer, scheduler, criterion, epochs, data_eval, file):
    for epoch in range(epochs):
        model.train()
        
        for i, (x, y) in enumerate(train_loader):
            x, y = x.to(device).to(torch.float32), y.to(device).to(torch.float32)
            optimizer.zero_grad()
            output = model(x)
            list_loss = [criterion[j](output[:, j], y[:, j]) for j in range(len(criterion))]
            # list_loss = [criterion[j](output[:, j], y[:, j]) for j in range(len(criterion)-1)] + \
            #     [10*criterion[-1](output[:, -1], y[:, -1])]
            loss = reduce(lambda x, y: x+y, list_loss)
            loss.backward()
            
            optimizer.step()
            if i % 100 == 0:
                str_loss = ', '.join(['loss{}: {:.6f}'.format(j, x.item()) for j, x in enumerate(list_loss)]) + ', loss: {:.6f}'.format(loss.item())
                logger.info('Epoch: [{}/{}], Step: [{}/{}], Lr: {:.6f}, '.format(
                    epoch+1, epochs, i+1, len(train_loader), optimizer.param_groups[0]['lr'])+str_loss)
        
        val(eval_loader, model, criterion, data_eval, file)
        
        scheduler.step()
        
        
def train_loss(train_loader, eval_loader, model, optimizer, scheduler, criterion_train, criterion_eval, epochs, data_eval, file):
    for epoch in range(epochs):
        model.train()
        
        for i, (x, y, w1, w2, w3) in enumerate(train_loader):
            x, y = x.to(device).to(torch.float32), y.to(device).to(torch.float32)
            w1, w2, w3 = w1.to(device).to(torch.float32), w2.to(device).to(torch.float32), w3.to(device).to(torch.float32)
            optimizer.zero_grad()
            output = model(x)
            list_loss = [criterion_train[j](output[:, j], y[:, j], w1) for j in [0, 1]] + \
                [criterion_train[2](output[:, 2], y[:, 2], w2)] + \
                [criterion_train[j](output[:, j], y[:, j], w3) for j in [3, 4]]
            loss = reduce(lambda x, y: x+y, list_loss)
            loss.backward()
            
            optimizer.step()
            if i % 100 == 0:
                str_loss = ', '.join(['loss{}: {:.6f}'.format(j, x.item()) for j, x in enumerate(list_loss)]) + ', loss: {:.6f}'.format(loss.item())
                logger.info('Epoch: [{}/{}], Step: [{}/{}], Lr: {:.6f}, '.format(
                    epoch+1, epochs, i+1, len(train_loader), optimizer.param_groups[0]['lr'])+str_loss)
        
        val_loss(eval_loader, model, criterion_eval, data_eval, file)
        
        scheduler.step()

In [ ]:
def val(eval_loader, model, criterion, data_eval, file):
    model.eval()
    
    global best_loss
    global best_auc
    y_true = data_eval[1]
    num_class = y_true.shape[1]
    
    eval_loss = 0
    list_eval_loss = [0] * num_class
    y_pred = []
    
    with torch.no_grad():
        for i, (x, y) in enumerate(eval_loader):
            x, y = x.to(device).to(torch.float32), y.to(device).to(torch.float32)
            output = model(x)
            list_eval_loss = [x+criterion[j](output[:, j], y[:, j]).item() for j, x in enumerate(list_eval_loss)]
            eval_loss = sum(list_eval_loss)
            
            output1 = torch.sigmoid(output)
            if i == 0:
                y_pred = output1.cpu().numpy()
            else:
                y_pred = np.concatenate((y_pred, output1.cpu().numpy()), axis=0)
    
    list_eval_loss = [x/len(eval_loader) for x in list_eval_loss]
    eval_loss /= len(eval_loader)
    
    eval_auc = [util.auc(y_true[:, i], y_pred[:, i]) for i in range(num_class)]
    str_auc = ', '.join(['AUC{}: {:.6f}'.format(i, eval_auc[i]) for i in range(num_class)])
    str_loss = 'Eval set: Average loss: {:.6f}, '.format(eval_loss) + ', '.join(['Average loss{}: {:.6f}'.format(j, x) for j, x in enumerate(list_eval_loss)])
    
    logger.info('{}, {}'.format(str_loss, str_auc))
    
    if list_eval_loss[-1] < best_loss: # 只关注链路尾部输出
        best_loss = list_eval_loss[-1]
        torch.save(model.state_dict(), './save/{}.pth'.format(file))
        model1 = torch.jit.script(model)
        torch.jit.save(model1, './save/{}.pt'.format(file))
        logger.info('Save model with loss: {:.6f}, {}'.format(best_loss, str_auc))
        

def val_loss(eval_loader, model, criterion, data_eval, file):
    model.eval()
    
    global best_loss
    global best_auc
    y_true = data_eval[1]
    num_class = y_true.shape[1]
    
    eval_loss = 0
    list_eval_loss = [0] * num_class
    y_pred = []
    
    with torch.no_grad():
        for i, (x, y, w1, w2, w3) in enumerate(eval_loader):
            x, y = x.to(device).to(torch.float32), y.to(device).to(torch.float32)
            w1, w2, w3 = w1.to(device).to(torch.float32), w2.to(device).to(torch.float32), w3.to(device).to(torch.float32)
            output = model(x)
            # list_eval_loss = [x+criterion[j](output[:, j], y[:, j], w1).item() for j, x in enumerate(list_eval_loss[:2])] + \
            #     [list_eval_loss[2]+criterion[2](output[:, 2], y[:, 2], w2).item()] + \
            #     [x+criterion[j](output[:, j], y[:, j], w3).item() for j, x in enumerate(list_eval_loss[3:])]
            list_eval_loss = [x+criterion[j](output[:, j], y[:, j]).item() for j, x in enumerate(list_eval_loss)]
            eval_loss = sum(list_eval_loss)
            
            output1 = torch.sigmoid(output)
            if i == 0:
                y_pred = output1.cpu().numpy()
            else:
                y_pred = np.concatenate((y_pred, output1.cpu().numpy()), axis=0)
    
    list_eval_loss = [x/len(eval_loader) for x in list_eval_loss]
    eval_loss /= len(eval_loader)
    
    eval_auc = [util.auc(y_true[:, i], y_pred[:, i]) for i in range(num_class)]
    str_auc = ', '.join(['AUC{}: {:.6f}'.format(i, eval_auc[i]) for i in range(num_class)])
    str_loss = 'Eval set: Average loss: {:.6f}, '.format(eval_loss) + ', '.join(['Average loss{}: {:.6f}'.format(j, x) for j, x in enumerate(list_eval_loss)])
    
    logger.info('{}, {}'.format(str_loss, str_auc))
    
    if list_eval_loss[2] + list_eval_loss[-1] < best_loss: # 关注链路特定位置的输出
        best_loss = list_eval_loss[2] + list_eval_loss[-1] # label_pre_pass, label_pass_1
        torch.save(model.state_dict(), './save/{}.pth'.format(file))
        model1 = torch.jit.script(model)
        torch.jit.save(model1, './save/{}.pt'.format(file))
        logger.info('Save model with loss: {:.6f}, {}'.format(best_loss, str_auc))

* 训练

In [ ]:
criterion_train = (BCEWL_weighted_multi_task().to(device), BCEWL_weighted_multi_task().to(device), BCEWL_weighted_multi_task().to(device), BCEWL_weighted_multi_task().to(device), BCEWL_weighted_multi_task().to(device))
criterion_eval = (torch.nn.BCEWithLogitsLoss().to(device), torch.nn.BCEWithLogitsLoss().to(device), torch.nn.BCEWithLogitsLoss().to(device), torch.nn.BCEWithLogitsLoss().to(device), torch.nn.BCEWithLogitsLoss().to(device))

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 20, 30, 40], gamma=0.1)
# scheduler_wu = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=5, after_scheduler=scheduler_ms)

best_loss = np.inf
best_auc = 0

# train_loss(train_loader, eval_loader, model, optimizer, scheduler, criterion_train, criterion_eval, 50, (df_X_train_eval_transform.values, df_y_train_eval.values), 'mmoe_condition_6_loss_pp_cb')
# train_loss(train_loader, eval_loader, model, optimizer, scheduler, criterion_train, criterion_eval, 50, (df_X_train_eval_transform.values, df_y_train_eval.values), 'mmoe_condition_6_loss_pp_cb_0')
# train_loss(train_loader, eval_loader, model, optimizer, scheduler, criterion_train, criterion_eval, 50, (df_X_train_eval_transform.values, df_y_train_eval.values), 'mmoe_condition_6_loss_pp_cb_0_filter')
train_loss(train_loader, eval_loader, model, optimizer, scheduler, criterion_train, criterion_eval, 50, (df_X_train_eval_transform.values, df_y_train_eval.values), 'mmoe_condition_6_loss_pp_cb_feats')

In [ ]:
def test(test_loader, model, criterion, data_eval):
    model.eval()
    
    y_true = data_eval[1]
    num_class = y_true.shape[1]
    
    eval_loss = 0
    list_eval_loss = [0] * num_class
    y_pred = []
    
    with torch.no_grad():
        for i, (x, y) in enumerate(test_loader):
            x, y = x.to(device).to(torch.float32), y.to(device).to(torch.float32)
            output = model(x)
            list_eval_loss = [x+criterion[j](output[:, j], y[:, j]).item() for j, x in enumerate(list_eval_loss)]
            eval_loss = sum(list_eval_loss)
            
            output1 = torch.sigmoid(output)
            if i == 0:
                y_pred = output1.cpu().numpy()
            else:
                y_pred = np.concatenate((y_pred, output1.cpu().numpy()), axis=0)
    
    list_eval_loss = [x/len(test_loader) for x in list_eval_loss]
    eval_loss /= len(test_loader)
    
    eval_auc = [util.auc(y_true[:, i], y_pred[:, i]) for i in range(num_class)]
    str_auc = ', '.join(['AUC{}: {:.6f}'.format(i, eval_auc[i]) for i in range(num_class)])
    str_loss = 'Test set: Average loss: {:.6f}, '.format(eval_loss) + ', '.join(['Average loss{}: {:.6f}'.format(j, x) for j, x in enumerate(list_eval_loss)])
    
    print('{}, {}'.format(str_loss, str_auc))
    return y_pred


def test_loss(test_loader, model, criterion, data_eval):
    model.eval()
    
    y_true = data_eval[1]
    num_class = y_true.shape[1]
    
    eval_loss = 0
    list_eval_loss = [0] * num_class
    y_pred = []
    
    with torch.no_grad():
        for i, (x, y, w1, w2, w3) in enumerate(test_loader):
            x, y = x.to(device).to(torch.float32), y.to(device).to(torch.float32)
            w1, w2, w3 = w1.to(device).to(torch.float32), w2.to(device).to(torch.float32), w3.to(device).to(torch.float32)
            output = model(x)
            # list_eval_loss = [x+criterion[j](output[:, j], y[:, j], w1).item() for j, x in enumerate(list_eval_loss[:2])] + \
            #     [list_eval_loss[2]+criterion[2](output[:, 2], y[:, 2], w2).item()] + \
            #     [x+criterion[j](output[:, j], y[:, j], w3).item() for j, x in enumerate(list_eval_loss[3:])]
            list_eval_loss = [x+criterion[j](output[:, j], y[:, j]).item() for j, x in enumerate(list_eval_loss)]
            eval_loss = sum(list_eval_loss)
            
            output1 = torch.sigmoid(output)
            if i == 0:
                y_pred = output1.cpu().numpy()
            else:
                y_pred = np.concatenate((y_pred, output1.cpu().numpy()), axis=0)
    
    list_eval_loss = [x/len(test_loader) for x in list_eval_loss]
    eval_loss /= len(test_loader)
    
    eval_auc = [util.auc(y_true[:, i], y_pred[:, i]) for i in range(num_class)]
    str_auc = ', '.join(['AUC{}: {:.6f}'.format(i, eval_auc[i]) for i in range(num_class)])
    
    # 长表
    # eval_auc = [util.auc(y_true[:, i], y_pred[:, i]) for i in [0, 1, 3, 4]]
    # str_auc = ', '.join(['AUC{}: {:.6f}'.format(i, eval_auc[i]) for i in range(4)])
    
    str_loss = 'Test set: Average loss: {:.6f}, '.format(eval_loss) + ', '.join(['Average loss{}: {:.6f}'.format(j, x) for j, x in enumerate(list_eval_loss)])
    
    print('{}, {}'.format(str_loss, str_auc))
    return y_pred

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
util.seed_everything(2023)

config = {
    'Model': {
        'num_experts': 7, 
        'expert_hidden_units': [512, 256, 128], 
        'units': 64, 
        'num_tasks': 5, 
        'tower_hidden_units': [64, 32, 16], 
        'dropout': 0.5, 
        'use_bn': False
    }
}

# feats_columns = utils_.load_pickle('../data/exp2suc/loss_pp_cb/feats_columns.pickle')
feats_columns = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/feats_columns.pickle')

model_rebuild = MMOE(config, feats_columns).to(device)

# model_rebuild.load_state_dict(torch.load('save/mmoe_condition_6_loss_pp_cb.pth'))
# model_rebuild.load_state_dict(torch.load('save/mmoe_condition_6_loss_pp_cb_0.pth'))
# model_rebuild.load_state_dict(torch.load('save/mmoe_condition_6_loss_pp_cb_0_filter.pth'))
model_rebuild.load_state_dict(torch.load('save/mmoe_condition_6_loss_pp_cb_feats.pth'))

model_rebuild

In [ ]:
# X_eval = utils_.load_pickle('../data/exp2suc/loss_pp_cb/df_X_pi_train_eval_transform_20230716_20231001.pickle')
# y_eval = utils_.load_pickle('../data/exp2suc/loss_pp_cb/df_y_train_eval_20230716_20231001.pickle')

X_eval = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/df_X_pi_train_eval_transform_20230716_20231001.pickle')
y_eval = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/df_y_train_eval_20230716_20231001.pickle')

print(X_eval.shape)
print(y_eval.shape)

# 权重
# 降低
w_pre_link_eval = X_eval[['item_id_4', 'item_id_5']].max(axis=1)
w_pre_link_eval = w_pre_link_eval.apply(lambda x: 5 if x == 0 else 1)
print(w_pre_link_eval.shape)

w_gf_pp_eval = X_eval[['item_id_0', 'table_type_1']].sum(axis=1)
w_gf_pp_eval = w_gf_pp_eval.apply(lambda x: 10 if x == 2 else 1)
print(w_gf_pp_eval.shape)

w_after_link_eval= X_eval[['item_id_0', 'item_id_4', 'item_id_5', 'table_type_1']].\
    apply(lambda x: 1 if x['item_id_0']==1 and x['table_type_1']==1 
                      else 2 if x['item_id_4']==1 or x['item_id_5']==1 
                             else 10, 
          axis=1)
print(w_after_link_eval.shape)

# 置0
# w_pre_link_eval = X_eval[['item_id_4', 'item_id_5']].max(axis=1)
# w_pre_link_eval = w_pre_link_eval.apply(lambda x: 5 if x == 0 else 1)
# print(w_pre_link_eval.shape)

# w_gf_pp_eval = X_eval[['item_id_0', 'table_type_1']].sum(axis=1)
# w_gf_pp_eval = w_gf_pp_eval.apply(lambda x: 1 if x == 2 else 0)
# print(w_gf_pp_eval.shape)

# w_after_link_eval= X_eval[['item_id_0', 'item_id_4', 'item_id_5', 'table_type_1']].\
#     apply(lambda x: 0 if x['item_id_0']==1 and x['table_type_1']==1 
#                       else 1 if x['item_id_4']==1 or x['item_id_5']==1 
#                              else 5, 
#           axis=1)
# print(w_after_link_eval.shape)

test_loader = DataLoader(
    DatasetLoader_www(X_eval.values, y_eval.values, 
                      w_pre_link_eval.values, w_gf_pp_eval.values, w_after_link_eval.values), 
    1024, shuffle=False, num_workers=8)

criterion = (torch.nn.BCEWithLogitsLoss().to(device), torch.nn.BCEWithLogitsLoss().to(device), torch.nn.BCEWithLogitsLoss().to(device), torch.nn.BCEWithLogitsLoss().to(device), torch.nn.BCEWithLogitsLoss().to(device))

y_pred = test_loss(test_loader, model_rebuild, criterion, (X_eval.values, y_eval.values))

In [ ]:
df_y = pd.concat([y_eval.reset_index(drop=True), pd.DataFrame(data=y_pred, columns=['pred_'+x for x in y_eval.columns])], axis=1)
print(df_y.shape)
df_y.head()

In [ ]:
def cal_recall(df_y, y_true, y_pred, threshold=0.3):
    df_y_sort = df_y.copy()
    df_y_sort.sort_values(by=y_pred, ascending=False, inplace=True)
    df_y_sort.reset_index(drop=True, inplace=True)
    
    df_y_sort['label_recall'] = 0
    df_y_sort.loc[:int(df_y_sort.shape[0]*threshold), 'label_recall'] = 1
    
    dict_cr = classification_report(df_y_sort[y_true], df_y_sort['label_recall'], output_dict=True)
    
    return dict_cr['1.0']['recall']

In [ ]:
try:
    with tqdm(y_eval.columns) as t:
        for x in t:
            print('{}: {}'.format(x, cal_recall(df_y[[x, 'pred_'+x]], x, 'pred_'+x, threshold=0.2)))
except KeyboardInterrupt:
    t.close()
    raise
t.close()

In [ ]:
# df_id_train_eval = utils_.load_pickle('../data/exp2suc/loss_pp_cb/df_id_train_eval_20230716_20231001.pickle')
df_id_train_eval = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/df_id_train_eval_20230716_20231001.pickle')

print(df_id_train_eval.shape)
df_id_train_eval.head()

In [ ]:
df_id_train_eval_ = df_id_train_eval.reset_index(drop=True)
df_id_train_eval_.head()

In [ ]:
recall_p = 20
list_tuple_item_table = [
    (0, 1), # aa短表
    (0, 2), # aa长表
    (1, 2), # bb长表
    (2, 2), # cc长表
    (3, 2), # dd长表
    (4, 2), # ee长表
    (5, 2)  # ff长表
]

try:
    with tqdm(list_tuple_item_table) as t:
        for item_id, table_type in t:
            print('item_id:{bi}, table_type:{tt}'.format(bi=item_id, tt=table_type))
            
            index_item_table = df_id_train_eval_[
                (df_id_train_eval_['item_id']==item_id)
                &(df_id_train_eval_['table_type']==table_type)].index
            df_y_item_table = df_y.loc[index_item_table, :]
            print(df_y_item_table.shape[0])
            
            list_label = y_eval.columns
            if table_type == 1:
                list_label = [x for x in list_label if x not in ['label_pass', 'label_pass_1']]
            else:
                list_label = [x for x in list_label if x not in ['label_pre_pass']]
                
            for x in list_label:
                if df_y_item_table[df_y_item_table[x]==1].shape[0] == 0:
                    continue
                print('{l}, AUC:{ras}, Recall@{p}%:{cr}'.format(
                    l=x, 
                    ras=roc_auc_score(df_y_item_table[x], df_y_item_table['pred_'+x]), 
                    p=recall_p, 
                    cr=cal_recall(df_y_item_table, x, 'pred_'+x, threshold=recall_p*0.01)))
except:
    t.close()
    raise
t.close()

* 测试

In [ ]:
df_fusion = pd.read_csv('../data/exp2suc/sample_label_feature_fusion_test_20230716_20231001.txt', sep='\t', encoding='utf-8')
df_asp1 = pd.read_csv('../data/exp2suc/sample_label_feature_aspiration_part1_test_20230716_20231001.txt', sep='\t', encoding='utf-8')
df_asp2 = pd.read_csv('../data/exp2suc/sample_label_feature_aspiration_part2_test_20230716_20231001.txt', sep='\t', encoding='utf-8')

print(df_fusion.shape)
print(df_asp1.shape)
print(df_asp2.shape)

In [ ]:
list_feats_id_dt_item_y = ['uid', 'obs_dt', 'dt', 'item', 'table_type', 'label_apply', 'label_submit', 'label_pre_pass', 'label_pass', 'label_pass_1']

list_feats_x_fusion = [x for x in 
                       utils_.load_pickle('../data/exp2suc/list_feats/list_feats_x_fusion_pi_20230716_20231001.pickle') 
                       if x not in 
                       ['pqg_last_7_day_didipay_use_ticket_discount_amount']
                      ]
list_feats_x_asp1 = [x for x in 
                     utils_.load_pickle('../data/exp2suc/list_feats/list_feats_x_asp1_pi_20230716_20231001.pickle') 
                     if x not in 
                     []
                    ]
list_feats_x_asp2 = [x for x in 
                     utils_.load_pickle('../data/exp2suc/list_feats/list_feats_x_asp2_pi_20230716_20231001.pickle') 
                     if x not in 
                     []
                    ]

print(len(list_feats_x_fusion))
print(len(list_feats_x_asp1))
print(len(list_feats_x_asp2))

In [ ]:
df = df_fusion[list_feats_id_dt_item_y+list_feats_x_fusion].\
    merge(df_asp1[list_feats_id_dt_item_y+list_feats_x_asp1]).\
    merge(df_asp2[list_feats_id_dt_item_y+list_feats_x_asp2])

print(df.shape)
df.head()

In [ ]:
df_des = utils_.df_des(df)
df_des.to_csv('../data/exp2suc/df_des_pi_test_20230716_20231001.csv', encoding='utf-8')

In [ ]:
dict_item = {
    'aaa': 0, 
    'bbb': 1, 
    'ccc': 2, 
    'ddd': 3, 
    'eee': 4, 
    'fff': 5
}

df['item_id'] = df['item']
df['item_id'].replace(dict_item, inplace=True)
df.head()

In [ ]:
df[['item', 'item_id', 'table_type', 'uid']].groupby(['item', 'item_id', 'table_type']).count()

In [ ]:
# 物料\表类型特征处理
# one hot encoding
list_feats_ohe = ['item_id', 'table_type']
list_df_ohe = []
try:
    with tqdm(list_feats_ohe) as t:
        for feat in t:
            df_ohe_feat = utils_.one_hot_encoder(df, feat)
            list_df_ohe.append(df_ohe_feat)
except KeyboardInterrupt:
    t.close()
    raise
t.close()

df_ohe_part = pd.concat(list_df_ohe, axis=1)
print(df_ohe_part.shape)
df_ohe_part.head()

In [ ]:
df_ohe = pd.concat([df, df_ohe_part], axis=1)
print(df_ohe.shape)
df_ohe.head()

In [ ]:
utils_.save_pickle(df_ohe, '../data/exp2suc/df_pi_ohe_test_20230716_20231001.pickle')

In [ ]:
df_ohe = utils_.load_pickle('../data/exp2suc/df_pi_ohe_test_20230716_20231001.pickle')
print(df_ohe.shape)
df_ohe.head()

In [ ]:
# condition_feats_item = utils_.load_pickle('../data/exp2suc/loss_pp_cb/list_condition_feats_item.pickle')
# condition_feats_table = utils_.load_pickle('../data/exp2suc/loss_pp_cb/list_condition_feats_table.pickle')
# sparse_feats = utils_.load_pickle('../data/exp2suc/loss_pp_cb/list_sparse_feats.pickle')
# dense_feats = utils_.load_pickle('../data/exp2suc/loss_pp_cb/list_dense_feats.pickle')

condition_feats_item = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/list_condition_feats_item.pickle')
condition_feats_table = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/list_condition_feats_table.pickle')
sparse_feats = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/list_sparse_feats.pickle')
dense_feats = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/list_dense_feats.pickle')

print(len(condition_feats_item))
print(len(condition_feats_table))
print(len(sparse_feats))
print(len(dense_feats))

In [ ]:
# 连续
# ss = joblib.load('../data/exp2suc/loss_pp_cb/ss.pickle')
ss = joblib.load('../data/exp2suc/loss_pp_cb_feats/ss.pickle')

X_dense_ss = ss.transform(df_ohe[dense_feats])
print(X_dense_ss.shape)

In [ ]:
df_X_dense = pd.DataFrame(X_dense_ss, columns=dense_feats)
print(df_X_dense.shape)
df_X_dense.head()

In [ ]:
# 离散
# dict_lbe = joblib.load('../data/exp2suc/loss_pp_cb/dict_lbe.pickle')
dict_lbe = joblib.load('../data/exp2suc/loss_pp_cb_feats/dict_lbe.pickle')

for x in sparse_feats:
    print(x, dict_lbe[x].classes_)

In [ ]:
list_X_sparse = []

try:
    with tqdm(sparse_feats) as t:
        for x in t:
            list_feat_values_unseen = list(set(df_ohe[x].unique())-set(dict_lbe[x].classes_))
            if len(list_feat_values_unseen) > 0:
                print(x)
                df_ohe[x].replace(list_feat_values_unseen, -1, inplace=True)
            df_X_sparse_each = pd.DataFrame(dict_lbe[x].transform(df_ohe[x]), columns=[x])
            list_X_sparse.append(df_X_sparse_each)
except KeyboardInterrupt:
    t.close()
    raise
t.close()

df_X_sparse = pd.concat(list_X_sparse, axis=1)
print(df_X_sparse.shape)
df_X_sparse.head()

In [ ]:
# 合并
df_X_condition_item = df_ohe[condition_feats_item]
df_X_condition_table = df_ohe[condition_feats_table]
df_X_transform = pd.concat([df_X_condition_item, df_X_condition_table, df_X_sparse, df_X_dense], axis=1)
print(df_X_transform.shape)
df_X_transform.head()

In [ ]:
# 权重
# 降低
w_pre_link_eval = df_X_transform[['item_id_4', 'item_id_5']].max(axis=1)
w_pre_link_eval = w_pre_link_eval.apply(lambda x: 5 if x == 0 else 1)
print(w_pre_link_eval.shape)

w_gf_pp_eval = df_X_transform[['item_id_0', 'table_type_1']].sum(axis=1)
w_gf_pp_eval = w_gf_pp_eval.apply(lambda x: 10 if x == 2 else 1)
print(w_gf_pp_eval.shape)

w_after_link_eval= df_X_transform[['item_id_0', 'item_id_4', 'item_id_5', 'table_type_1']].\
    apply(lambda x: 1 if x['item_id_0']==1 and x['table_type_1']==1 
                      else 2 if x['item_id_4']==1 or x['item_id_5']==1 
                             else 10, 
          axis=1)
print(w_after_link_eval.shape)

# 置0
# w_pre_link_eval = df_X_transform[['item_id_4', 'item_id_5']].max(axis=1)
# w_pre_link_eval = w_pre_link_eval.apply(lambda x: 5 if x == 0 else 1)
# print(w_pre_link_eval.shape)

# w_gf_pp_eval = df_X_transform[['item_id_0', 'table_type_1']].sum(axis=1)
# w_gf_pp_eval = w_gf_pp_eval.apply(lambda x: 1 if x == 2 else 0)
# print(w_gf_pp_eval.shape)

# w_after_link_eval= df_X_transform[['item_id_0', 'item_id_4', 'item_id_5', 'table_type_1']].\
#     apply(lambda x: 0 if x['item_id_0']==1 and x['table_type_1']==1 
#                       else 1 if x['item_id_4']==1 or x['item_id_5']==1 
#                              else 5, 
#           axis=1)
# print(w_after_link_eval.shape)

In [ ]:
y = df_ohe[['label_apply', 'label_submit', 'label_pre_pass', 'label_pass', 'label_pass_1']]
print(y.shape)
y.head()

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
util.seed_everything(2023)

config = {
    'Model': {
        'num_experts': 7, 
        'expert_hidden_units': [512, 256, 128], 
        'units': 64, 
        'num_tasks': 5, 
        'tower_hidden_units': [64, 32, 16], 
        'dropout': 0.5, 
        'use_bn': False
    }
}

# feats_columns = utils_.load_pickle('../data/exp2suc/loss_pp_cb/feats_columns.pickle')
feats_columns = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/feats_columns.pickle')

model = MMOE(config, feats_columns).to(device)

# model.load_state_dict(torch.load('save/mmoe_condition_6_loss_pp_cb.pth'))
# model.load_state_dict(torch.load('save/mmoe_condition_6_loss_pp_cb_0.pth'))
# model.load_state_dict(torch.load('save/mmoe_condition_6_loss_pp_cb_0_filter.pth'))
model.load_state_dict(torch.load('save/mmoe_condition_6_loss_pp_cb_feats.pth'))

model

In [ ]:
test_loader = DataLoader(
    DatasetLoader_www(df_X_transform.values, y.values, 
                      w_pre_link_eval.values, w_gf_pp_eval.values, w_after_link_eval.values), 
    1024, shuffle=False, num_workers=8)

criterion = (torch.nn.BCEWithLogitsLoss().to(device), torch.nn.BCEWithLogitsLoss().to(device), torch.nn.BCEWithLogitsLoss().to(device), torch.nn.BCEWithLogitsLoss().to(device), torch.nn.BCEWithLogitsLoss().to(device))

y_pred = test_loss(test_loader, model, criterion, (df_X_transform.values, y.values))

In [ ]:
df_y = pd.concat([y, pd.DataFrame(data=y_pred, columns=['pred_'+x for x in y.columns])], axis=1)
print(df_y.shape)
df_y.head()

In [ ]:
try:
    with tqdm(y.columns) as t:
        for x in t:
            print('{}: {}'.format(x, cal_recall(df_y[[x, 'pred_'+x]], x, 'pred_'+x, threshold=0.2)))
except KeyboardInterrupt:
    t.close()
    raise
t.close()

In [ ]:
df_id = df_ohe[['uid', 'obs_dt', 'dt', 'item', 'item_id', 'table_type']]
df_id.head()

In [ ]:
# 分物料、表类型
recall_p = 20
list_tuple_item_table = [
    (0, 1), # aa短表
    (0, 2), # aa长表
    (1, 2), # bb长表
    (2, 2), # cc长表
    (3, 2), # dd长表
    (4, 2), # ee长表
    (5, 2)  # ff长表
]

try:
    with tqdm(list_tuple_item_table) as t:
        for item_id, table_type in t:
            print('item_id:{bi}, table_type:{tt}'.format(bi=item_id, tt=table_type))
            
            index_item_table = df_id[
                (df_id['item_id']==item_id)
                &(df_id['table_type']==table_type)].index
            df_y_item_table = df_y.loc[index_item_table, :]
            print(df_y_item_table.shape[0])
            
            list_label = y.columns
            if table_type == 1:
                list_label = [x for x in list_label if x not in ['label_pass', 'label_pass_1']]
            else:
                list_label = [x for x in list_label if x not in ['label_pre_pass']]
                
            for x in list_label:
                if df_y_item_table[df_y_item_table[x]==1].shape[0] == 0:
                    continue
                print('{l}, AUC:{ras}, Recall@{p}%:{cr}'.format(
                    l=x, 
                    ras=roc_auc_score(df_y_item_table[x], df_y_item_table['pred_'+x]), 
                    p=recall_p, 
                    cr=cal_recall(df_y_item_table, x, 'pred_'+x, threshold=recall_p*0.01)))
except:
    t.close()
    raise
t.close()

* feature importance

In [ ]:
# shap
shap_explainer = shap.DeepExplainer(model, [df_X_transform.values])
shap_explainer

In [ ]:
shap_values = shap_explainer.shap_values([df_X_transform.values])
shap_values

In [ ]:
shap.summary_plot(shap_values, df_train_X, plot_type='bar')

## 模型比较，oot

In [ ]:
df_fusion = pd.read_csv('../data/exp2suc/sample_label_feature_fusion_pi_20231008.txt', sep='\t', encoding='utf-8')
df_asp1 = pd.read_csv('../data/exp2suc/sample_label_feature_aspiration_part1_pi_20231008.txt', sep='\t', encoding='utf-8')
df_asp2 = pd.read_csv('../data/exp2suc/sample_label_feature_aspiration_part2_pi_20231008.txt', sep='\t', encoding='utf-8')

print(df_fusion.shape)
print(df_asp1.shape)
print(df_asp2.shape)

In [ ]:
list_feats_id_dt_item_y = ['uid', 'obs_dt', 'dt', 'item', 'table_type', 'label_apply', 'label_submit', 'label_pre_pass', 'label_pass', 'label_pass_1']

list_feats_x_fusion = [x for x in 
                       utils_.load_pickle('../data/exp2suc/list_feats/list_feats_x_fusion_pi_20230716_20231001.pickle') 
                       if x not in 
                       ['xxx']
                      ]
list_feats_x_asp1 = [x for x in 
                     utils_.load_pickle('../data/exp2suc/list_feats/list_feats_x_asp1_pi_20230716_20231001.pickle') 
                     if x not in 
                     []
                    ]
list_feats_x_asp2 = [x for x in 
                     utils_.load_pickle('../data/exp2suc/list_feats/list_feats_x_asp2_pi_20230716_20231001.pickle') 
                     if x not in 
                     []
                    ]

print(len(list_feats_x_fusion))
print(len(list_feats_x_asp1))
print(len(list_feats_x_asp2))

In [ ]:
df = df_fusion[list_feats_id_dt_item_y+list_feats_x_fusion].\
    merge(df_asp1[list_feats_id_dt_item_y+list_feats_x_asp1]).\
    merge(df_asp2[list_feats_id_dt_item_y+list_feats_x_asp2])

print(df.shape)
df.head()

In [ ]:
df_des = utils_.df_des(df)
df_des.to_csv('../data/exp2suc/df_des_pi_20231008.csv', encoding='utf-8')

In [ ]:
dict_item = {
    'aaa': 0, 
    'bbb': 1, 
    'ccc': 2, 
    'ddd': 3, 
    'eee': 4, 
    'fff': 5
}

df['item_id'] = df['item']
df['item_id'].replace(dict_item, inplace=True)
df.head()

In [ ]:
df[['item', 'item_id', 'table_type', 'uid']].groupby(['item', 'item_id', 'table_type']).count()

In [ ]:
# 物料\表类型特征处理
# one hot encoding
list_feats_ohe = ['item_id', 'table_type']
list_df_ohe = []
try:
    with tqdm(list_feats_ohe) as t:
        for feat in t:
            df_ohe_feat = utils_.one_hot_encoder(df, feat)
            list_df_ohe.append(df_ohe_feat)
except KeyboardInterrupt:
    t.close()
    raise
t.close()

df_ohe_part = pd.concat(list_df_ohe, axis=1)
print(df_ohe_part.shape)
df_ohe_part.head()

In [ ]:
df_ohe = pd.concat([df, df_ohe_part], axis=1)
print(df_ohe.shape)
df_ohe.head()

In [ ]:
utils_.save_pickle(df_ohe, '../data/exp2suc/df_pi_ohe_20231008.pickle')

In [ ]:
df_ohe = utils_.load_pickle('../data/exp2suc/df_pi_ohe_20231008.pickle')
print(df_ohe.shape)
df_ohe.head()

In [ ]:
# 长表，4物料
df_ohe = df_ohe[
    (df_ohe['table_type']==2)
    # &(df_ohe['item'].isin(['aaa', 'bbb', 'ccc','ddd']))
]
df_ohe.reset_index(drop=True, inplace=True)
print(df_ohe.shape)

In [ ]:
# condition_feats_item = utils_.load_pickle('../data/exp2suc/loss_pp_cb/list_condition_feats_item.pickle')
# condition_feats_table = utils_.load_pickle('../data/exp2suc/loss_pp_cb/list_condition_feats_table.pickle')
# sparse_feats = utils_.load_pickle('../data/exp2suc/loss_pp_cb/list_sparse_feats.pickle')
# dense_feats = utils_.load_pickle('../data/exp2suc/loss_pp_cb/list_dense_feats.pickle')

condition_feats_item = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/list_condition_feats_item.pickle')
condition_feats_table = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/list_condition_feats_table.pickle')
sparse_feats = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/list_sparse_feats.pickle')
dense_feats = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/list_dense_feats.pickle')

print(len(condition_feats_item))
print(len(condition_feats_table))
print(len(sparse_feats))
print(len(dense_feats))

In [ ]:
# 连续
# ss = joblib.load('../data/exp2suc/loss_pp_cb/ss.pickle')
ss = joblib.load('../data/exp2suc/loss_pp_cb_feats/ss.pickle')

X_dense_ss = ss.transform(df_ohe[dense_feats])
print(X_dense_ss.shape)

In [ ]:
df_X_dense = pd.DataFrame(X_dense_ss, columns=dense_feats)
print(df_X_dense.shape)
df_X_dense.head()

In [ ]:
# 离散
# dict_lbe = joblib.load('../data/exp2suc/loss_pp_cb/dict_lbe.pickle')
dict_lbe = joblib.load('../data/exp2suc/loss_pp_cb_feats/dict_lbe.pickle')

for x in sparse_feats:
    print(x, dict_lbe[x].classes_)

In [ ]:
list_X_sparse = []

try:
    with tqdm(sparse_feats) as t:
        for x in t:
            list_feat_values_unseen = list(set(df_ohe[x].unique())-set(dict_lbe[x].classes_))
            if len(list_feat_values_unseen) > 0:
                print(x)
                df_ohe[x].replace(list_feat_values_unseen, -1, inplace=True)
            df_X_sparse_each = pd.DataFrame(dict_lbe[x].transform(df_ohe[x]), columns=[x])
            list_X_sparse.append(df_X_sparse_each)
except KeyboardInterrupt:
    t.close()
    raise
t.close()

df_X_sparse = pd.concat(list_X_sparse, axis=1)
print(df_X_sparse.shape)
df_X_sparse.head()

In [ ]:
# 合并
df_X_condition_item = df_ohe[condition_feats_item]
df_X_condition_table = df_ohe[condition_feats_table]
df_X_transform = pd.concat([df_X_condition_item, df_X_condition_table, df_X_sparse, df_X_dense], axis=1)
print(df_X_transform.shape)
df_X_transform.head()

In [ ]:
# 权重
# 降低
w_pre_link_eval = df_X_transform[['item_id_4', 'item_id_5']].max(axis=1)
w_pre_link_eval = w_pre_link_eval.apply(lambda x: 5 if x == 0 else 1)
print(w_pre_link_eval.shape)

w_gf_pp_eval = df_X_transform[['item_id_0', 'table_type_1']].sum(axis=1)
w_gf_pp_eval = w_gf_pp_eval.apply(lambda x: 10 if x == 2 else 1)
print(w_gf_pp_eval.shape)

w_after_link_eval= df_X_transform[['item_id_0', 'item_id_4', 'item_id_5', 'table_type_1']].\
    apply(lambda x: 1 if x['item_id_0']==1 and x['table_type_1']==1 
                      else 2 if x['item_id_4']==1 or x['item_id_5']==1 
                             else 10, 
          axis=1)
print(w_after_link_eval.shape)

# 置0
# w_pre_link_eval = df_X_transform[['item_id_4', 'item_id_5']].max(axis=1)
# w_pre_link_eval = w_pre_link_eval.apply(lambda x: 5 if x == 0 else 1)
# print(w_pre_link_eval.shape)

# w_gf_pp_eval = df_X_transform[['item_id_0', 'table_type_1']].sum(axis=1)
# w_gf_pp_eval = w_gf_pp_eval.apply(lambda x: 1 if x == 2 else 0)
# print(w_gf_pp_eval.shape)

# w_after_link_eval= df_X_transform[['item_id_0', 'item_id_4', 'item_id_5', 'table_type_1']].\
#     apply(lambda x: 0 if x['item_id_0']==1 and x['table_type_1']==1 
#                       else 1 if x['item_id_4']==1 or x['item_id_5']==1 
#                              else 5, 
#           axis=1)
# print(w_after_link_eval.shape)

In [ ]:
y = df_ohe[['label_apply', 'label_submit', 'label_pre_pass', 'label_pass', 'label_pass_1']]
print(y.shape)
y.head()

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
util.seed_everything(2023)

config = {
    'Model': {
        'num_experts': 7, 
        'expert_hidden_units': [512, 256, 128], 
        'units': 64, 
        'num_tasks': 5, 
        'tower_hidden_units': [64, 32, 16], 
        'dropout': 0.5, 
        'use_bn': False
    }
}

# feats_columns = utils_.load_pickle('../data/exp2suc/loss_pp_cb/feats_columns.pickle')
feats_columns = utils_.load_pickle('../data/exp2suc/loss_pp_cb_feats/feats_columns.pickle')

model = MMOE(config, feats_columns).to(device)

# model.load_state_dict(torch.load('save/mmoe_condition_6_loss_pp_cb.pth'))
# model.load_state_dict(torch.load('save/mmoe_condition_6_loss_pp_cb_0.pth'))
# model.load_state_dict(torch.load('save/mmoe_condition_6_loss_pp_cb_0_filter.pth'))
model.load_state_dict(torch.load('save/mmoe_condition_6_loss_pp_cb_feats.pth'))

model

In [ ]:
test_loader = DataLoader(
    DatasetLoader_www(
        df_X_transform.values, y.values, 
        w_pre_link_eval.values, w_gf_pp_eval.values, w_after_link_eval.values), 
    1024, shuffle=False, num_workers=8)

criterion = (torch.nn.BCEWithLogitsLoss().to(device), torch.nn.BCEWithLogitsLoss().to(device), torch.nn.BCEWithLogitsLoss().to(device), torch.nn.BCEWithLogitsLoss().to(device), torch.nn.BCEWithLogitsLoss().to(device))

y_pred = test_loss(test_loader, model, criterion, (df_X_transform.values, y.values))

In [ ]:
df_y = pd.concat([y, pd.DataFrame(data=y_pred, columns=['pred_'+x for x in y.columns])], axis=1)
print(df_y.shape)
df_y.head()

In [ ]:
try:
    with tqdm(y.columns) as t:
        for x in t:
            print('{}: {}'.format(x, cal_recall(df_y[[x, 'pred_'+x]], x, 'pred_'+x, threshold=0.2)))
except KeyboardInterrupt:
    t.close()
    raise
t.close()

In [ ]:
cal_recall(df_y[['label_pass_1', 'pred_label_pass_1']], 'label_pass_1', 'pred_label_pass_1', threshold=0.2)

In [ ]:
df_id = df_ohe[['uid', 'obs_dt', 'dt', 'item', 'item_id', 'table_type']]
df_id.head()

In [ ]:
# 分物料、表类型
recall_p = 20
list_tuple_item_table = [
    (0, 1), # aa短表
    (0, 2), # aa长表
    (1, 2), # bb长表
    (2, 2), # cc长表
    (3, 2), # dd长表
    (4, 2), # ee长表
    (5, 2)  # ff长表
]

try:
    with tqdm(list_tuple_item_table) as t:
        for item_id, table_type in t:
            print('item_id:{bi}, table_type:{tt}'.format(bi=item_id, tt=table_type))
            
            index_item_table = df_id[
                (df_id['item_id']==item_id)
                &(df_id['table_type']==table_type)].index
            df_y_item_table = df_y.loc[index_item_table, :]
            print(df_y_item_table.shape[0])
            
            list_label = y.columns
            if table_type == 1:
                list_label = [x for x in list_label if x not in ['label_pass', 'label_pass_1']]
            else:
                list_label = [x for x in list_label if x not in ['label_pre_pass']]
                
            for x in list_label:
                if df_y_item_table[df_y_item_table[x]==1].shape[0] == 0:
                    continue
                print('{l}, AUC:{ras}, Recall@{p}%:{cr}'.format(
                    l=x, 
                    ras=roc_auc_score(df_y_item_table[x], df_y_item_table['pred_'+x]), 
                    p=recall_p, 
                    cr=cal_recall(df_y_item_table, x, 'pred_'+x, threshold=recall_p*0.01)))
except:
    t.close()
    raise
t.close()